In [26]:
import pandas as pd
import os
import random
import numpy as np

In [131]:
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# 데이터 경로 맞춰주세요!
DATA_PATH = '/opt/ml/input/data/train_dataset/train_data.csv'
df = pd.read_csv(DATA_PATH, dtype=dtype, parse_dates=['Timestamp'])
df = df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [133]:
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# 데이터 경로 맞춰주세요!
test_csv_file_path = '/opt/ml/input/data/train_dataset/test_data.csv'
test_df = pd.read_csv(test_csv_file_path, dtype=dtype, parse_dates=['Timestamp'])
test_df = test_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)
test_df = test_df[test_df['answerCode']!= -1].copy()

In [132]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


In [134]:
test_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623
...,...,...,...,...,...,...
260108,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244


In [135]:
df = pd.concat([df,test_df])

In [136]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
260108,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244


In [137]:
def percentile(s):
    return np.sum(s) / len(s)

In [138]:
correct_t = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
correct_t.columns = ["test_mean", 'test_sum']
correct_k = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
correct_k.columns = ["tag_mean", 'tag_sum']

In [139]:
df = pd.merge(df, correct_t, on=['testId'], how="left")
df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")

In [140]:
diff = df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
diff = diff.fillna(pd.Timedelta(seconds=0))
diff = diff['Timestamp'].apply(lambda x: x.total_seconds())

# 푸는 시간
df['elapsed'] = diff
df['elapsed'] = df['elapsed'].apply(lambda x : x if x <650 and x >=0 else None)
# 앞 세자리 
# 중간 세자리
# 문제 번호
df['grade']=df['testId'].apply(lambda x : int(x[1:4])//10)
df['mid'] = df['testId'].apply(lambda x : int(x[-3:]))
df['problem_number'] = df['assessmentItemID'].apply(lambda x : int(x[-3:]))

In [141]:
df.to_csv("/opt/ml/input/data/DB/.csv")

### 학생 DB
- user_elapsed : 평균 푸는 시간
- user_acc : 평균 정답률
- answer_rate_* : grade * 정답률 (None or Numeric)

In [143]:
stu_groupby = df.groupby('userID').agg({
    'answerCode': [percentile,'sum','count']
})
stu_groupby['user_acc'] = stu_groupby[('answerCode','sum')]/stu_groupby[('answerCode','count')]
stu_groupby

answerCode               user_acc
       percentile    sum count          
userID                                  
0        0.630872  470.0   745  0.630872
1        0.853162  796.0   933  0.853162
2        0.612319  169.0   276  0.612319
3        0.692754  717.0  1035  0.692754
4        0.694030  465.0   670  0.694030
...           ...    ...   ...       ...
7437     0.375000    6.0    16  0.375000
7438     0.750000   12.0    16  0.750000
7439     0.733333   11.0    15  0.733333
7440     0.400000    6.0    15  0.400000
7441     0.555556    5.0     9  0.555556

[7442 rows x 4 columns]

In [144]:
G = df.groupby(['userID','grade']).agg({
    'answerCode':['mean', 'count'],
    'elapsed':['mean']
})
G.reset_index(inplace=True)
G

userID grade answerCode           elapsed
                         mean count        mean
0          0     2   0.722222    36   37.206897
1          0     6   0.791908   346   36.533784
2          0     8   0.468320   363   39.261981
3          1     4   0.822719   581   96.679204
4          1     9   0.903409   352  103.672297
...      ...   ...        ...   ...         ...
19734   7439     4   0.733333    15   38.384615
19735   7440     3   0.400000    10   29.125000
19736   7440     5   0.400000     5   18.600000
19737   7441     3   0.200000     5   44.000000
19738   7441     4   1.000000     4   43.333333

[19739 rows x 5 columns]

In [145]:
G.columns = ['userID','grade','acc','count','elapsed']

In [146]:
G.to_csv("/opt/ml/input/data/DB/student_grade.csv")

### 시험지 DB
- testId :
- elapsed : 평균 푸는 시간
- problem_number : 문제 수
- type_number : 유형 수
- grade : 앞 세자리
- mid : 뒤 세자리

In [147]:
f = lambda x : len(set(x))

test_df = df.groupby(['testId']).agg({
    'elapsed': 'mean',
    'answerCode':['mean', 'sum'],
    'problem_number':'max',
    'KnowledgeTag':f
})
test_df.reset_index(inplace=True)
test_df['grade']=test_df['testId'].apply(lambda x : int(x[1:4])//10)
test_df['mid']=test_df['testId'].apply(lambda x : int(x[-3:]))

test_df.head(20)

testId    elapsed answerCode         problem_number KnowledgeTag  \
                     mean       mean     sum            max     <lambda>   
0   A010000001  33.949286   0.923429  1616.0              5            1   
1   A010000002  34.218902   0.931429  1630.0              5            2   
2   A010000003  36.326211   0.842857  1475.0              5            3   
3   A010000004  34.215713   0.880571  1541.0              5            3   
4   A010000005  36.883227   0.849143  1486.0              5            2   
5   A010000006  41.751037   0.846286  1481.0              5            2   
6   A010000007  47.975658   0.822286  1439.0              5            1   
7   A010000008  40.276280   0.896571  1569.0              5            1   
8   A010000009  64.125957   0.814286  1425.0              5            1   
9   A010000010  52.338367   0.842857  1475.0              5            1   
10  A010000011  73.481690   0.725714  1270.0              5            1   
11  A010000012  72.928273   0.546857   957.0              5            1   
12  A010000013  66.339943   0.820571  1436.0              5            1   
13  A010000014  42.846841   0.878286  1537.0              5            1   
14  A010000015  90.991412   0.753571  1055.0              5            1   
15  A010000016  40.226389   0.872571  1527.0              5            1   
16  A010000017  50.522593   0.812500  2275.0              8            1   
17  A010000018  56.537059   0.875714  2452.0              8            1   
18  A010000019  89.457343   0.736571  1289.0              5            1   
19  A010000020  57.317181   0.877857  1229.0              4            1   

   grade mid  
              
0      1   1  
1      1   2  
2      1   3  
3      1   4  
4      1   5  
5      1   6  
6      1   7  
7      1   8  
8      1   9  
9      1  10  
10     1  11  
11     1  12  
12     1  13  
13     1  14  
14     1  15  
15     1  16  
16     1  17  
17     1  18  
18     1  19  
19     1  20

In [148]:
test_df.columns = ['testId','elapsed','acc','count','problem_number',"type_number","grade","mid"]

In [149]:
test_df.to_csv("/opt/ml/input/data/DB/test_info.csv")

### 문제별 DB

### Knowledge Tag 별 DB

In [ ]:
correct_k = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
correct_k.columns = ["tag_mean", 'tag_sum']

In [ ]:
print(f"""--- BASIC INFORMATIONS ---
userID           : {df.userID.nunique()}
assessmentItemID : {df.assessmentItemID.nunique()}
testID           : {df.testId.nunique()}
mean answer rate : {df.answerCode.sum() / df.shape[0] * 100:.2f}%
KnowledgeTag     : {df.KnowledgeTag.nunique()}
{'-'*26}""")